In [1]:
import numpy as np
import open3d as o3d
import copy
import csv
from matplotlib import pyplot as plt
from open3d import visualization

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#FOR TUM DATASET
x = [] #dvs pixel x in mm
y = [] #dvs pixel y in mm
depth = [] #depth from depth sensor
time_stamp = [] 
x_pixel = [] #dvs pixel x in mm
y_pixel = [] #dvs pixel y 


#events.tsv: A TSV file where each row corresponds to one event.
# Provided data is: PrimeSense image coordinates (x,y), PrimeSense depth measurement,
# EDVS image coordinates (x,y), Timestamp (in microsenconds), EDVS parity flag

with open('/home/bru/Dev/DATA/scn2c_take01/take01/events.csv', encoding="Latin-1") as csvfile:  #change this according to your file 
    reader = list(csv.reader(csvfile, delimiter=","))

num_events = len(reader)
num_events_batch = 5000
import pandas as pd

for i in range(0,num_events):
    depth.append((reader[i][2]))
    x.append(((int(reader[i][3])/128)-0.5)*1000) # convert x_pixels to x in millimeters (according to camera intrinsics of DAVIS128) K = [128, 0, 64; 0, 128, 64; 0, 0, 1]
    y.append(((int(reader[i][4])/128)-0.5)*1000) # convert y_pixels to y in millimeters
    x_pixel.append(int(reader[i][3])) 
    y_pixel.append(int(reader[i][4]))
    time_stamp.append((reader[i][5]))
depth = np.array(depth).astype(np.float64)
Max_depth = depth.max()
Min_depth = depth.min()
x = np.array(x)
x = x-x.min()
y = np.array(y)
y = y-y.min()


In [3]:
"""
import numpy as np

# Assuming this is your intrinsic matrix
K = np.array([
    [128, 0, 64],  # fx, 0, cx
    [0, 128, 64],  # 0, fy, cy
    [0, 0, 1]  # 0, 0, 1
])

# Invert K
K_inv = np.linalg.inv(K)

# Assume we have pixel coordinates (u, v) and the corresponding depth Z
for i in range(0,num_events):
    u = int(reader[i][3])
    v = int(reader[i][4])
    Z = int(reader[i][2])

    X, Y, _ = np.dot(K_inv, np.array([u, v, 1])) * Z
    x.append(X)
    y.append(Y)
"""

'\nimport numpy as np\n\n# Assuming this is your intrinsic matrix\nK = np.array([\n    [128, 0, 64],  # fx, 0, cx\n    [0, 128, 64],  # 0, fy, cy\n    [0, 0, 1]  # 0, 0, 1\n])\n\n# Invert K\nK_inv = np.linalg.inv(K)\n\n# Assume we have pixel coordinates (u, v) and the corresponding depth Z\nfor i in range(0,num_events):\n    u = int(reader[i][3])\n    v = int(reader[i][4])\n    Z = int(reader[i][2])\n\n    X, Y, _ = np.dot(K_inv, np.array([u, v, 1])) * Z\n    x.append(X)\n    y.append(Y)\n'

In [4]:
iEv = 0 #num event
iBatch = 0 #num batch
data_size = (num_events_batch ,3) 
num_arr = 0
while True:
    # Check if we have processed all events
    if (iEv + num_events_batch > num_events):
        break
        
    # Select subset of events or data
    iEv += num_events_batch
    iBatch += 1
    
    depth_batch = np.array(depth[iEv+0:iEv+num_events_batch]).astype(np.float64)
    x_batch = np.array(x[iEv+0:iEv+num_events_batch]).astype(np.float64)
    y_batch = np.array(y[iEv+0:iEv+num_events_batch]).astype(np.float64)
    print(depth_batch.min(),depth_batch.max())
    #batches = (np.c_[x_batch,y_batch, depth_batch]).astype(np.float64)
    #print(batches)
    x_max = int(x_batch.max())
    y_max = int(y_batch.max())
    if y_max < x_max:
        img = np.zeros((int(x_batch.max()+1),int(x_batch.max()+1)))
        count = np.zeros((int(x_batch.max()+1),int(x_batch.max()+1))) 
    else:
        img = np.zeros((int(y_batch.max()+1),int(y_batch.max()+1)))
        count = np.zeros((int(y_batch.max()+1),int(y_batch.max()+1)))
        
        for i in range(num_events_batch):
        # Need to convert the polarity bit from {0,1} to {-1,+1} and accumulate
            img[int(y_batch[i]),int(x_batch[i])] += (int(depth_batch[i]))
            count[int(y_batch[i]),int(x_batch[i])] += 1

    # Compute per-pixel average if count at the pixel is >1
        count[count < 1] = 1  # to avoid division by zero
        img = img / count*2
    
        print(img.shape)             
        plt.imshow(img,cmap='jet_r',clim=(Min_depth,Max_depth))
        plt.show()

        
    iEv += num_events_batch
    iBatch += 1
   

1307.0 4463.0
1283.0 4463.0
1214.0 4405.0
1149.0 4295.0
1073.0 4241.0
1007.0 4139.0
920.0 3994.0
858.0 3948.0
811.0 3948.0
773.0 3859.0
747.0 3732.0
762.0 3692.0
761.0 3576.0
803.0 3333.0
807.0 3300.0
796.0 3300.0
762.0 3238.0
583.0 3238.0
744.0 3238.0
762.0 3238.0
792.0 3238.0
798.0 3269.0
821.0 3333.0
843.0 3333.0
843.0 3300.0
819.0 3269.0
805.0 3238.0
802.0 3238.0
807.0 3238.0
813.0 3238.0
823.0 3238.0
835.0 3238.0
833.0 3208.0
807.0 3208.0
510.0 3149.0
783.0 3120.0
811.0 3149.0
831.0 3178.0
847.0 3208.0
849.0 3208.0
856.0 3178.0
864.0 3120.0
896.0 3238.0
962.0 3399.0
998.0 3433.0
1007.0 3399.0
1031.0 3433.0
1084.0 3503.0
1094.0 3576.0
1115.0 3539.0
1134.0 3503.0
1141.0 3539.0
1141.0 3539.0
1138.0 3576.0
1141.0 3576.0
789.0 3539.0
1130.0 3576.0
1126.0 3614.0
1126.0 3614.0
1123.0 3614.0
1138.0 3576.0
1145.0 3576.0
1161.0 3503.0
658.0 3503.0
1161.0 3503.0
1153.0 3503.0
1138.0 3503.0
1115.0 3433.0
1108.0 3399.0
1101.0 3399.0
1090.0 3433.0
1090.0 3365.0
1105.0 3433.0
1108.0 3433.0
683.0

In [5]:
 
depth_batch = np.array(depth[iEv+0:iEv+num_events_batch]).astype(np.float64)
x_batch = np.array(x[iEv+0:iEv+num_events_batch]).astype(np.float64)
y_batch = np.array(y[iEv+0:iEv+num_events_batch]).astype(np.float64)
   
        
    
batches = (np.c_[x_batch,y_batch, depth_batch]).astype(np.float64)
    #print(batches)
    
#x_batch= x_batch-x_batch.min()
x_max = int(x_batch.max())
#y_batch= y_batch-y_batch.min()
y_max = int(y_batch.max())
if y_max < x_max:
    img = np.zeros((int(x_batch.max()+1),int(x_batch.max()+1)))
else:
    img = np.zeros((int(y_batch.max()+1),int(y_batch.max()+1)))
                      
for i in range(num_events_batch):
    img[int(y_batch[i]),int(x_batch[i])] += (int(depth_batch[i]))
plt.imshow(img, clim=(0,50))
plt.show()
print(img.shape)
 

IndexError: index 4609 is out of bounds for axis 0 with size 4609

In [6]:
x_pos = []
y_pos = []
z_pos = []
x_rot = []
y_rot = []
z_rot = []
T_rest = np.identity(4)
T_new = []
T_orj = []

num_batch_apart = 1 #skip this much point cloud (When point clouds are too close to each other, there is not much change)
#Can be dynamically change!!

for i in range(iBatch-num_batch_apart):
    source = o3d.io.read_point_cloud("/home/bru/Dev/DATA/scn2c_take01/take01/point_clouds/depth_event_%08d.ply" %i)
    target = o3d.io.read_point_cloud("/home/bru/Dev/DATA/scn2c_take01/take01/point_clouds/depth_event_%08d.ply" %(i+num_batch_apart))
    
    max_correspondence_distance = 30   # mm, for ICP
    trans_init = np.identity(4) #for ICP to initialize transformation matrix might be changed to see how robust the algorithm is??
    
    evaluation = o3d.pipelines.registration.evaluate_registration(source, target,
                                                    max_correspondence_distance, trans_init)
          
    #POINT TO POINT ICP
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=3000))
    
    # The class TransformationEstimationPointToPoint provides functions to compute the residuals and Jacobian matrices of the point-to-point ICP objective. 
    # The function registration_icp takes it as a parameter and runs point-to-point ICP to obtain the results.
  
    T = reg_p2p.transformation
    T_orj.append(T) # list of delta T, transformations from one scene to another
   
    T_rest = np.dot(T_rest, T)
print(T_rest)
    #T_new.append(T_rest)
    

[[ 9.78757280e-01 -2.01487564e-01  3.79070908e-02  1.92088400e+01]
 [ 1.93542272e-01  9.69023673e-01  1.53409615e-01 -4.44943593e+02]
 [-6.76429979e-02 -1.42814153e-01  9.87435336e-01 -4.72967256e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [7]:
source = o3d.io.read_point_cloud("/home/bru/Dev/DATA/scn2c_take01/take01/point_clouds/depth_event_%08d.ply" %0)
target = o3d.io.read_point_cloud("/home/bru/Dev/DATA/scn2c_take01/take01/point_clouds/depth_event_%08d.ply" %195)

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw([source_temp, target_temp])
    
#ilename = "/home/bru/Dev/DATA/scn2c_take01/reg_res%08d.png" %1
    
draw_registration_result(source, target, T_rest)

max_correspondence_distance = 30   # mm, for ICP
evaluation = o3d.pipelines.registration.evaluate_registration(source, target,
                                                    max_correspondence_distance, T_rest)

print(evaluation)

[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.
[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.


KeyboardInterrupt: 

In [ ]:
import SimpleITK as sitk

img = sitk.PhysicalPointSource(img)
transform = sitk.AffineTransform(T_rest.flatten(), img.GetDimension())

# Apply the transformation
resampler = sitk.ResampleImageFilter()
resampler.SetTransform(transform)
resampler.SetSize(image.GetSize())
resampler.SetOutputOrigin(img.GetOrigin())
resampler.SetOutputSpacing([x/y for x,y in zip(img.GetSpacing(), transform_matrix.diagonal())])
resampler.SetOutputDirection(image.GetDirection())
resampled_image = resampler.Execute(img)